# Import packages

In [ ]:
# # if using google drive
# %cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate, KFold
)

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



# Predict curvature

## explanatory_var

In [ ]:
from decision_making_analysis import free_selection, replacement, trajectory_info

In [ ]:
reload(free_selection)

In [ ]:
raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330'

moit = intended_targets_classes.ModelOfIntendedTargets(raw_data_folder_path=raw_data_folder_path, time_range_of_trajectory=[-0.5, 2.5], num_time_points_for_trajectory=10)
moit.get_monkey_data(already_retrieved_ok=True, include_ff_dataframe=True)
moit.ff_dataframe = moit.ff_dataframe[abs(moit.ff_dataframe['ff_angle_boundary']) <= math.pi/4]
moit.ff_dataframe = moit.ff_dataframe[moit.ff_dataframe['time_since_last_vis'] <= 2.5]

# moit.get_and_process_manual_anno_long(n_seconds_after_crossing_boundary=2.5)
moit.make_curvature_df([-25, 25], curv_of_traj_mode='distance')

moit.make_auto_annot()
moit.manual_anno = moit.auto_annot
moit.manual_anno_long = moit.auto_annot_long


moit.eliminate_crossing_boundary_cases(n_seconds_after_crossing_boundary=0.5)
moit.get_input_data(num_ff_per_row=5, keeping_1_out_of_n_rows=10, 
                    add_arc_info=True, curvature_df=moit.curvature_df, curv_of_traj_df=moit.curv_of_traj_df.copy()
                    )

explanatory_var = moit.free_selection_x_df.drop(columns=['point_index'], errors='ignore').values

In [ ]:
reload(pattern_by_trials)
reload(decision_making_class)
reload(decision_making_utils)
reload(show_null_trajectory)
reload(intended_targets_classes)
reload(regression_utils)
reload(curvature_utils)

## dependent_var

In [ ]:
# get dependent variables
reload(curvature_utils)
all_point_index = moit.chosen_rows_of_df.starting_point_index.values
curv_of_traj_df, traj_curv_descr = curv_of_traj_utils.find_curv_of_traj_df_based_on_curv_of_traj_mode(moit.window_for_curv_of_traj, moit.monkey_information, 
                                                                                                      moit.ff_caught_T_new, curv_of_traj_mode=moit.curv_of_traj_mode, truncate_curv_of_traj_by_time_of_capture=moit.truncate_curv_of_traj_by_time_of_capture)
all_point_index = moit.free_selection_x_df['point_index'].values
dependent_var = curv_of_traj_df.set_index('point_index').loc[all_point_index, 'curv_of_traj'].values

In [ ]:
curv_of_traj_df.column

## train test split

In [ ]:
keep_whole_chunk = False
test_size = 0.2

scaler = StandardScaler()
dependent_var_sc = scaler.fit_transform(dependent_var.reshape(-1,1)) #scale data
explanatory_var_sc = scaler.fit_transform(explanatory_var)


if keep_whole_chunk:
    indices = np.arange(len(dependent_var_sc))
    num_test_points = int(len(indices)*test_size)
    num_train_points = len(indices)-num_test_points
    # make sure that the test chunk will be a whole segment in the data, to minimize the splitting up of the train and test chunks
    test_indice_start = np.random.randint(0, num_train_points)
    indices_test = indices[test_indice_start:test_indice_start+num_test_points]
    indices_train = np.setdiff1d(indices, indices_test)
    X_train = explanatory_var_sc[indices_train]
    X_test = explanatory_var_sc[indices_test]
    y_train = dependent_var_sc[indices_train]
    y_test = dependent_var_sc[indices_test]
else:
    X_train, X_test, y_train, y_test = train_test_split(explanatory_var_sc, dependent_var_sc, test_size=test_size, random_state=42)

## ML

In [ ]:
# use MLPRegressor
mlp = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
mlp.score(X_test, y_test)

In [ ]:
# use KNeighborsRegressor
n_neighbors = int(math.sqrt(len(y_train)))
neigh = KNeighborsRegressor(n_neighbors=n_neighbors)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

In [ ]:
# use RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

In [ ]:
# use linear regression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)